<img src="https://relevance.ai/wp-content/uploads/2021/11/logo.79f303e-1.svg" width="150" alt="Relevance AI" />
<h5> Developer-first vector platform for ML teams </h5>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RelevanceAI/workflows/blob/main/workflows/taxonomy/Taxonomy.ipynb)

Inserting your own taxonomy or labels with Relevance AI allows users to better understand.

In [ ]:
#@title After filling this form, press the top left button.
# You can grab your token here https://cloud.relevance.ai/sdk/api

import base64
import json

token = "" #@param {type:"string"}

show_warnings_in_logs = False #@param {type:"boolean"}
#@markdown Once the form is filled and you've clicked run, monitor below for logs of it running

config = json.loads(base64.b64decode(token + "==="))
token = config['authorizationToken']
dataset_id = config['dataset_id']
taxonomy_labels = config['taxonomy_labels']
fields = config['fields']
threshold = config['similarity_threshold']
max_number_of_labels = int(config['max_number_of_labels'])
output_field = config.get("output_field")

#@markdown <h3>Optional Configurations</h3>
model_id = "sentence-transformers/all-mpnet-base-v2" #@param  {type:"string"}

taxonomy_labels = [x for x in taxonomy_labels if x.strip() != ""]

!pip install -q -U RelevanceAI==2.3.5
!pip install -q sentence-transformers==2.2.0

from relevanceai import Client
client = Client(token=token)
ds = client.Dataset(dataset_id)
from sentence_transformers import SentenceTransformer
enc = SentenceTransformer(model_id)

label_documents = [{"label": l, "label_vector_": enc.encode(l).tolist()} for l in taxonomy_labels]
for field in fields:
  
  if not field.endswith("_vector_"):
    ds.vectorize_text(fields=[field], models=[model_id])

  if not field.endswith("_vector_"):
    # rename to a different vector 
    field = field + "_" + model_id + "_vector_"
  vector_field = field.replace("/", "_")
  ds.label(
      vector_fields=[vector_field],
      label_documents=label_documents,
      max_number_of_labels=max_number_of_labels,
      output_field=output_field,
      similarity_threshold=threshold
  )